In [1]:
import numpy as np
import pandas as pd

In [2]:
def create_column_filter(df):
    new_df = df.copy()
    all_columns = new_df.columns
    saved_columns = []
    
    for column_name in new_df.columns:
        if column_name == 'CLASS':
            continue
        column_value = new_df.loc[:, column_name]
        values = []
        for item in column_value:
            if pd.isna(item):
                continue
            else:
                values.append(item)
        if len(set(values)) > 1 or len(values) == 1:
            saved_columns.append(column_name)
            
    deleted_colums = list(set(all_columns) ^ set(saved_columns))
    deleted_colums.remove('CLASS')
    
    for item in deleted_colums:
        new_df.pop(item)
    return new_df, saved_columns

In [3]:
def apply_column_filter(df, column_filter):
    new_df = df.copy()
    all_columns = list(new_df.columns)
    all_columns.remove('CLASS')
    
    remained_colums = list(set(all_columns) ^ set(column_filter))
    for item in remained_colums:
        new_df.pop(item)
    return new_df

In [4]:
# Test your code (leave this part unchanged)

df = pd.DataFrame({"CLASS":[1,0,1,0,1],"A":[1,2,np.nan,4,5],"B":[1,1,1,1,np.nan],"C":["h","h",np.nan,"i","h"],"D":[np.nan,np.nan,np.nan,np.nan,np.nan]})

filtered_df, column_filter = create_column_filter(df)

new_df = pd.DataFrame({"CLASS":[1,0,0],"A":[4,5,6],"B":[1,2,1],"C":[np.nan,np.nan,np.nan],"D":[np.nan,4,5]})

filtered_new_df = apply_column_filter(new_df,column_filter)

display("df",df)
display("filtered_df",filtered_df)
display("new_df",new_df)
display("filtered_new_df",filtered_new_df)

'df'

,CLASS,A,B,C,D
0,1,1.0,1.0,h,NaN
1,0,2.0,1.0,h,NaN
2,1,NaN,1.0,NaN,NaN
3,0,4.0,1.0,i,NaN
4,1,5.0,NaN,h,NaN


'filtered_df'

,CLASS,A,C
0,1,1.0,h
1,0,2.0,h
2,1,NaN,NaN
3,0,4.0,i
4,1,5.0,h


'new_df'

,CLASS,A,B,C,D
0,1,4,1,NaN,NaN
1,0,5,2,NaN,4.0
2,0,6,1,NaN,5.0


'filtered_new_df'

,CLASS,A,C
0,1,4,NaN
1,0,5,NaN
2,0,6,NaN


In [5]:
anneal_train_df = pd.read_csv("anneal_train.csv")
anneal_test_df = pd.read_csv("anneal_test.csv")
anneal_train_df

,family,product-type,steel,carbon,hardness,temper_rolling,condition,formability,strength,non-ageing,...,s,p,shape,thick,width,len,oil,bore,packing,CLASS
0,NaN,C,A,0,0,NaN,S,2.0,0,NaN,...,NaN,NaN,SHEET,0.999,1220.0,4880,NaN,0,NaN,3
1,NaN,C,NaN,0,0,NaN,A,2.0,0,NaN,...,NaN,NaN,SHEET,0.700,1320.0,4880,NaN,0,NaN,3
2,TN,C,A,0,0,NaN,NaN,3.0,0,N,...,NaN,NaN,SHEET,0.500,1220.0,4880,NaN,0,NaN,5
3,ZS,C,A,0,50,T,NaN,NaN,0,NaN,...,NaN,NaN,SHEET,0.451,1250.0,762,NaN,0,NaN,3
4,NaN,C,A,0,85,T,NaN,NaN,0,NaN,...,NaN,NaN,COIL,4.000,1000.0,0,NaN,600,NaN,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,ZS,C,A,0,70,T,NaN,NaN,0,NaN,...,NaN,NaN,COIL,0.600,610.0,0,NaN,0,NaN,3
445,NaN,C,NaN,0,0,NaN,S,1.0,0,NaN,...,NaN,NaN,COIL,0.300,609.9,0,NaN,0,NaN,3
446,NaN,C,K,45,0,NaN,NaN,NaN,0,NaN,...,NaN,NaN,COIL,2.300,900.0,0,NaN,600,NaN,3
447,NaN,C,A,0,0,NaN,S,3.0,0,N,...,NaN,NaN,SHEET,1.000,1320.0,4880,NaN,0,NaN,3


In [6]:
anneal_train_df.columns

Index(['family', 'product-type', 'steel', 'carbon', 'hardness',
       'temper_rolling', 'condition', 'formability', 'strength', 'non-ageing',
       'surface-finish', 'surface-quality', 'enamelability', 'bc', 'bf', 'bt',
       'bw/me', 'bl', 'm', 'chrom', 'phos', 'cbond', 'marvi', 'exptl', 'ferro',
       'corr', 'blue-bright-varn-clean', 'lustre', 'jurofm', 's', 'p', 'shape',
       'thick', 'width', 'len', 'oil', 'bore', 'packing', 'CLASS'],
      dtype='object')

In [7]:
def create_imputation(dataframe):
    
    df = dataframe.copy()
    imputation = {}
    
    for col in df.columns:
        
        if col != "ID" and col != "CLASS":  
            
            if df[col].dtypes == "int" or df[col].dtype == "float":
                
                df[col].fillna(df[col].mean(),inplace=True)
                imputation[col] = (df[col].mean())
                
            else:
                df[col].fillna(df[col].mode()[0],inplace=True)
                imputation[col] = (df[col].mode()[0])
        
    return df, imputation


In [8]:
# Test your code (leave this part unchanged)

anneal_train_df = pd.read_csv("anneal_train.csv")
anneal_test_df = pd.read_csv("anneal_test.csv")
#print(anneal_train_df.head())

anneal_train_imp, imputation = create_imputation(anneal_train_df)

display(anneal_train_imp)

,family,product-type,steel,carbon,hardness,temper_rolling,condition,formability,strength,non-ageing,...,s,p,shape,thick,width,len,oil,bore,packing,CLASS
0,TN,C,A,0,0,T,S,2.000000,0,N,...,NaN,NaN,SHEET,0.999,1220.0,4880,Y,0,3.0,3
1,TN,C,A,0,0,T,A,2.000000,0,N,...,NaN,NaN,SHEET,0.700,1320.0,4880,Y,0,3.0,3
2,TN,C,A,0,0,T,S,3.000000,0,N,...,NaN,NaN,SHEET,0.500,1220.0,4880,Y,0,3.0,5
3,ZS,C,A,0,50,T,S,2.251748,0,N,...,NaN,NaN,SHEET,0.451,1250.0,762,Y,0,3.0,3
4,TN,C,A,0,85,T,S,2.251748,0,N,...,NaN,NaN,COIL,4.000,1000.0,0,Y,600,3.0,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,ZS,C,A,0,70,T,S,2.251748,0,N,...,NaN,NaN,COIL,0.600,610.0,0,Y,0,3.0,3
445,TN,C,A,0,0,T,S,1.000000,0,N,...,NaN,NaN,COIL,0.300,609.9,0,Y,0,3.0,3
446,TN,C,K,45,0,T,S,2.251748,0,N,...,NaN,NaN,COIL,2.300,900.0,0,Y,600,3.0,3
447,TN,C,A,0,0,T,S,3.000000,0,N,...,NaN,NaN,SHEET,1.000,1320.0,4880,Y,0,3.0,3


In [9]:
anneal_train_df = pd.read_csv("anneal_train.csv")
anneal_test_df = pd.read_csv("anneal_test.csv")
display(anneal_test_df["family"].mode())

anneal_train_imp, imputation = create_imputation(anneal_train_df)
display(anneal_train_imp, imputation)

0    TN
dtype: object

,family,product-type,steel,carbon,hardness,temper_rolling,condition,formability,strength,non-ageing,...,s,p,shape,thick,width,len,oil,bore,packing,CLASS
0,TN,C,A,0,0,T,S,2.000000,0,N,...,NaN,NaN,SHEET,0.999,1220.0,4880,Y,0,3.0,3
1,TN,C,A,0,0,T,A,2.000000,0,N,...,NaN,NaN,SHEET,0.700,1320.0,4880,Y,0,3.0,3
2,TN,C,A,0,0,T,S,3.000000,0,N,...,NaN,NaN,SHEET,0.500,1220.0,4880,Y,0,3.0,5
3,ZS,C,A,0,50,T,S,2.251748,0,N,...,NaN,NaN,SHEET,0.451,1250.0,762,Y,0,3.0,3
4,TN,C,A,0,85,T,S,2.251748,0,N,...,NaN,NaN,COIL,4.000,1000.0,0,Y,600,3.0,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,ZS,C,A,0,70,T,S,2.251748,0,N,...,NaN,NaN,COIL,0.600,610.0,0,Y,0,3.0,3
445,TN,C,A,0,0,T,S,1.000000,0,N,...,NaN,NaN,COIL,0.300,609.9,0,Y,0,3.0,3
446,TN,C,K,45,0,T,S,2.251748,0,N,...,NaN,NaN,COIL,2.300,900.0,0,Y,600,3.0,3
447,TN,C,A,0,0,T,S,3.000000,0,N,...,NaN,NaN,SHEET,1.000,1320.0,4880,Y,0,3.0,3


{'family': 'TN',
 'product-type': 'C',
 'steel': 'A',
 'carbon': 3.859688195991091,
 'hardness': 13.084632516703786,
 'temper_rolling': 'T',
 'condition': 'S',
 'formability': 2.2517482517482557,
 'strength': 26.302895322939868,
 'non-ageing': 'N',
 'surface-finish': 'P',
 'surface-quality': 'E',
 'enamelability': 1.7142857142857018,
 'bc': 'Y',
 'bf': 'Y',
 'bt': 'Y',
 'bw/me': 'B',
 'bl': 'Y',
 'm': nan,
 'chrom': 'C',
 'phos': 'P',
 'cbond': 'Y',
 'marvi': nan,
 'exptl': nan,
 'ferro': 'Y',
 'corr': nan,
 'blue-bright-varn-clean': 'B',
 'lustre': 'Y',
 'jurofm': nan,
 's': nan,
 'p': nan,
 'shape': 'SHEET',
 'thick': 1.1911937639198227,
 'width': 769.4917594654789,
 'len': 1229.293986636971,
 'oil': 'Y',
 'bore': 35.18930957683742,
 'packing': 3.0}

In [10]:
def create_bins(dataframe,nobins=10,bintype="equal-width"):
    
    df = dataframe.copy()
    binning = {}
    for col in df.columns:
        if col != "CLASS" and col != "ID" and df[col].dtype in ["float64", "float32", "int64", "int32"]:
            if bintype == "equal-width":
                df[col], bins = pd.cut(df[col],nobins,retbins=True,duplicates="drop",labels=False)
                binning[col] = bins    
            elif bintype == "equal-size":
                df[col], bins = pd.qcut(df[col],q=nobins,retbins=True,duplicates="drop",labels=False)
                binning[col] = bins
            df[col] = df[col].astype("category")
            df[col] = df[col].cat.set_categories([str(i) for i in df[col].cat.categories], rename = True)
            binning[col][0] = -np.inf
            binning[col][-1] = np.inf
        else:
            df[col] = df[col].astype('category')
    
    return df, binning

In [11]:
glass_train_df = pd.read_csv("glass_train.csv")
display(glass_train_df)
glass_train_disc, binning = create_bins(glass_train_df,nobins=10,bintype="equal-size")
display(glass_train_disc)
display(binning)

,ID,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,CLASS
0,202,1.51653,11.95,0.00,1.19,75.18,2.70,8.93,0.00,0.00,7
1,124,1.51707,13.48,3.48,1.71,72.52,0.62,7.99,0.00,0.00,2
2,152,1.52127,14.32,3.90,0.83,71.50,0.00,9.49,0.00,0.00,3
3,197,1.51556,13.87,0.00,2.54,73.23,0.14,9.41,0.81,0.01,7
4,144,1.51709,13.00,3.47,1.79,72.72,0.66,8.18,0.00,0.00,2
...,...,...,...,...,...,...,...,...,...,...,...
102,178,1.51937,13.79,2.41,1.19,72.76,0.00,9.77,0.00,0.00,6
103,160,1.51796,13.50,3.36,1.63,71.94,0.57,8.81,0.00,0.09,3
104,88,1.51645,13.40,3.49,1.52,72.65,0.67,8.08,0.00,0.10,2
105,98,1.51743,12.20,3.25,1.16,73.55,0.62,8.90,0.00,0.24,2


,ID,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,CLASS
0,202,3,0,0,2,9,8,6,0,0,7
1,124,4,5,4,7,3,6,1,0,0,2
2,152,8,8,7,0,0,0,7,0,0,3
3,197,0,7,0,9,8,1,7,1,0,7
4,144,4,2,3,7,3,7,2,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...
102,178,7,6,1,2,4,0,8,0,0,6
103,160,6,6,2,7,1,4,5,0,1,3
104,88,2,5,4,5,3,7,1,0,1,2
105,98,4,0,2,1,9,6,6,0,2,2


{'RI': array([    -inf, 1.515896, 1.51618 , 1.516516, 1.516866, 1.51753 ,
        1.517902, 1.518618, 1.520114, 1.521846,      inf]),
 'Na': array([  -inf, 12.73 , 12.872, 13.   , 13.222, 13.38 , 13.492, 13.794,
        14.198, 14.82 ,    inf]),
 'Mg': array([ -inf, 1.82 , 3.188, 3.41 , 3.476, 3.55 , 3.61 , 3.728,   inf]),
 'Al': array([ -inf, 0.906, 1.172, 1.23 , 1.348, 1.48 , 1.54 , 1.622, 1.808,
        2.094,   inf]),
 'Si': array([  -inf, 71.756, 72.196, 72.388, 72.72 , 72.79 , 72.966, 73.06 ,
        73.208, 73.372,    inf]),
 'K': array([ -inf, 0.006, 0.148, 0.39 , 0.54 , 0.576, 0.6  , 0.636, 0.67 ,
          inf]),
 'Ca': array([  -inf,  7.978,  8.112,  8.338,  8.554,  8.67 ,  8.81 ,  9.032,
         9.674, 10.924,    inf]),
 'Ba': array([-inf, 0.78,  inf]),
 'Fe': array([ -inf, 0.062, 0.118, 0.24 ,   inf])}

In [12]:
train_df = pd.read_csv("tic-tac-toe_train.csv")
train_df

,top-left-square,top-middle-square,top-right-square,middle-left-square,middle-middle-square,middle-right-square,bottom-left-square,bottom-middle-square,bottom-right-square,CLASS
0,o,o,o,x,x,b,o,x,x,negative
1,b,b,o,x,x,o,x,b,o,negative
2,b,x,o,x,o,x,o,x,o,negative
3,o,b,b,o,b,b,x,x,x,positive
4,b,o,x,x,x,x,b,o,o,positive
...,...,...,...,...,...,...,...,...,...,...
474,b,o,b,x,x,x,o,b,b,positive
475,o,x,x,o,o,o,x,b,x,negative
476,o,o,x,o,b,x,x,b,x,positive
477,x,x,x,x,o,b,o,b,o,positive


In [22]:
def create_one_hot(dataframe):
    
    df = dataframe.copy()
    df_new = df.copy()
    one_hot = {}
    for col in df.columns:
        if col != "CLASS" and col != "ID":  
            if str(df.dtypes[col]) == "category" or str(df.dtypes[col]) == "object":
                df[col] = df[col].astype("category")
               # one_hot[col] = df[col].cat.categories
                one_hot[col] = list(df[col].cat.categories)
                for i in one_hot[col]:
                    name = col + "_" + str(i)  
                    # print(name)
                    new_col = df[col]==i
                    new_col = new_col.astype("float")
                    df_new[name] = new_col 
                df_new = df_new.drop(columns = col, axis = 1) 
                
    return df_new, one_hot

def split(dataframe, testfraction=0.5):
    
    df = dataframe.copy()
    
    df_random = df.reindex(np.random.permutation(df.index))
    
    trainingdf = df_random[0: int((1-testfraction)*df.shape[0])]
    testdf = df_random[int((1-testfraction)*df.shape[0])+1 : df.shape[0]]
    
    return trainingdf, testdf

In [23]:
train_df = pd.read_csv("tic-tac-toe_train.csv")
new_train, one_hot = create_one_hot(train_df)
test_df = pd.read_csv("tic-tac-toe_test.csv")
new_test_df = apply_one_hot(test_df,one_hot)
display("new_test_df",new_test_df)

,top-left-square,top-middle-square,top-right-square,middle-left-square,middle-middle-square,middle-right-square,bottom-left-square,bottom-middle-square,bottom-right-square,CLASS
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive
...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,negative
954,o,x,o,x,x,o,x,o,x,negative
955,o,x,o,x,o,x,x,o,x,negative
956,o,x,o,o,x,x,x,o,x,negative


,CLASS,top-left-square_b,top-left-square_o,top-left-square_x,top-middle-square_b,top-middle-square_o,top-middle-square_x,top-right-square_b,top-right-square_o,top-right-square_x,...,middle-right-square_x,bottom-left-square_b,bottom-left-square_o,bottom-left-square_x,bottom-middle-square_b,bottom-middle-square_o,bottom-middle-square_x,bottom-right-square_b,bottom-right-square_o,bottom-right-square_x
429,positive,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
245,positive,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
88,positive,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
62,positive,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
313,positive,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,positive,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
621,positive,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
661,negative,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
475,positive,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


{'top-left-square': ['b', 'o', 'x'],
 'top-middle-square': ['b', 'o', 'x'],
 'top-right-square': ['b', 'o', 'x'],
 'middle-left-square': ['b', 'o', 'x'],
 'middle-middle-square': ['b', 'o', 'x'],
 'middle-right-square': ['b', 'o', 'x'],
 'bottom-left-square': ['b', 'o', 'x'],
 'bottom-middle-square': ['b', 'o', 'x'],
 'bottom-right-square': ['b', 'o', 'x']}